Question 3
# DECISION TREE

# Connect Drive to Colab Notebook


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/SMAI/Assignment 1/q3')
!ls -l

total 469
-rw------- 1 root root  13370 Jan 26 07:23 data_description.txt
-rw------- 1 root root 143467 Jan 26 07:23 test.csv
-rw------- 1 root root   5941 Jan 26 07:23 test_labels.csv
-rw------- 1 root root 316335 Jan 26 07:23 train.csv


Imports for the task

In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

'''
For the Report metric
'''
from sklearn import tree
from scipy.stats.mstats import mquantiles
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support, mean_squared_error, median_absolute_error, mean_absolute_error
import seaborn as sb
from scipy import stats
import matplotlib.pyplot as plt

import math
import random
from collections import defaultdict, Counter

from pprint import pprint

In [0]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
try:
  df_train = df_train.drop('Id', axis=1)
  df_test = df_test.drop('Id', axis=1)
except:
  pass


# Data Desciption

df.describe() gives the decription of the data like max min standard deviation mean etc

In [0]:
df_train.head()

MSSubClass MSZoning  LotFrontage  ...  SaleType SaleCondition SalePrice
0          60       RL         99.0  ...        WD        Normal    354000
1          20       RL         89.0  ...        WD        Normal    378500
2         120       RM         32.0  ...        WD        Normal    153500
3         160       FV         30.0  ...       New       Partial    173733
4          20       RL          NaN  ...        WD        Normal    127000

[5 rows x 80 columns]

In [0]:
df_test.head()

MSSubClass MSZoning  LotFrontage  ...  YrSold SaleType SaleCondition
0          20       RL         80.0  ...    2010       WD        Normal
1          20       RL         60.0  ...    2008       WD        Normal
2          20       RL         93.0  ...    2008       WD        Normal
3          90       RL         64.0  ...    2010       WD        Normal
4          60       RL         65.0  ...    2009       WD        Normal

[5 rows x 79 columns]

In [0]:
df_train.describe()

MSSubClass  LotFrontage  ...       YrSold      SalePrice
count  1000.000000   825.000000  ...  1000.000000    1000.000000
mean     55.435000    71.310303  ...  2007.793000  182854.640000
std      41.477779    24.185834  ...     1.329631   82290.996386
min      20.000000    21.000000  ...  2006.000000   34900.000000
25%      20.000000    60.000000  ...  2007.000000  130000.000000
50%      50.000000    70.000000  ...  2008.000000  163950.000000
75%      70.000000    81.000000  ...  2009.000000  215000.000000
max     190.000000   313.000000  ...  2010.000000  755000.000000

[8 rows x 37 columns]

In [0]:
df_test.describe()

MSSubClass  LotFrontage  ...      MoSold       YrSold
count  460.000000   376.000000  ...  460.000000   460.000000
mean    60.076087    67.284574  ...    6.330435  2007.865217
std     43.914463    24.304214  ...    2.743093     1.324845
min     20.000000    21.000000  ...    1.000000  2006.000000
25%     20.000000    55.000000  ...    5.000000  2007.000000
50%     50.000000    65.000000  ...    6.000000  2008.000000
75%     70.000000    79.000000  ...    8.000000  2009.000000
max    190.000000   313.000000  ...   12.000000  2010.000000

[8 rows x 36 columns]

In [0]:
cols = []
numerical_features = df_train.dtypes[df_train.dtypes != 'object'].index
categorical_features = df_train.dtypes[df_train.dtypes == 'object'].index

numerical_features = numerical_features.drop('SalePrice')

print('Numerical Features', numerical_features.sort_values(), sep='\n')
print('*'*100) 
print('Categorical Features', categorical_features.sort_values(), sep='\n')

Numerical Features
Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars',
       'GarageYrBlt', 'GrLivArea', 'HalfBath', 'KitchenAbvGr', 'LotArea',
       'LotFrontage', 'LowQualFinSF', 'MSSubClass', 'MasVnrArea', 'MiscVal',
       'MoSold', 'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea',
       'ScreenPorch', 'TotRmsAbvGrd', 'TotalBsmtSF', 'WoodDeckSF', 'YearBuilt',
       'YearRemodAdd', 'YrSold'],
      dtype='object')
****************************************************************************************************
Categorical Features
Index(['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Fence', 'FireplaceQu', 'Foundation', 'Function

## Deal with NaN

For Categorical Data

In [0]:
_categorical_missing_values_train = df_train[categorical_features].isnull().sum().sort_values(ascending=False)
_categorical_missing_values_test = df_test[categorical_features].isnull().sum().sort_values(ascending=False)

_categorical_missing_values_train = _categorical_missing_values_train[_categorical_missing_values_train > 0]

_categorical_missing_values_test = _categorical_missing_values_test[_categorical_missing_values_test > 0]

print('Categorical Train missing Valuse ', _categorical_missing_values_train, sep='\n')

print('*'*100)

print('Categorical Test missing Values ', _categorical_missing_values_test, sep='\n')


Categorical Train missing Valuse 
PoolQC          994
MiscFeature     964
Alley           950
Fence           808
FireplaceQu     464
GarageCond       53
GarageQual       53
GarageFinish     53
GarageType       53
BsmtFinType2     31
BsmtExposure     31
BsmtFinType1     30
BsmtQual         30
BsmtCond         30
MasVnrType        7
Electrical        1
dtype: int64
****************************************************************************************************
Categorical Test missing Values 
PoolQC          459
MiscFeature     442
Alley           419
Fence           371
FireplaceQu     226
GarageCond       28
GarageQual       28
GarageFinish     28
GarageType       28
BsmtFinType1      7
BsmtFinType2      7
BsmtQual          7
BsmtCond          7
BsmtExposure      7
MasVnrType        1
dtype: int64


Fill None in Categorical Features if the the data cell is empty fill with None

In [0]:
for col in _categorical_missing_values_train.index:
    df_train[col].fillna('None', inplace=True)

for col in _categorical_missing_values_test.index:
    df_test[col].fillna('None', inplace = True)

'''
After filling with None Categorical Data in test and Train
'''

_categorical_missing_values_train = df_train[categorical_features].isnull().sum().sort_values(ascending=False)
_categorical_missing_values_test = df_test[categorical_features].isnull().sum().sort_values(ascending=False)

_categorical_missing_values_train = _categorical_missing_values_train[_categorical_missing_values_train > 0]

_categorical_missing_values_test = _categorical_missing_values_test[_categorical_missing_values_test > 0]

print('Categorical Train missing Valuse ', _categorical_missing_values_train, sep='\n')

print('*'*100)

print('Categorical Test missing Values ', _categorical_missing_values_test, sep='\n')


Categorical Train missing Valuse 
Series([], dtype: int64)
****************************************************************************************************
Categorical Test missing Values 
Series([], dtype: int64)


# Handling Numerical Nulls
Handling numerical features with empty cells

In [0]:
_numerical_missing_values_train = df_train[numerical_features].isnull().sum().sort_values(ascending=False)
_numerical_missing_values_test = df_test[numerical_features].isnull().sum().sort_values(ascending=False)

_numerical_missing_values_train = _numerical_missing_values_train[_numerical_missing_values_train > 0]

_numerical_missing_values_test = _numerical_missing_values_test[_numerical_missing_values_test > 0]

print('Numerical Train missing Valuse ', _numerical_missing_values_train, sep='\n')

print('*'*100)

print('Numerical Test missing Values ', _numerical_missing_values_test, sep='\n')


Numerical Train missing Valuse 
LotFrontage    175
GarageYrBlt     53
MasVnrArea       7
dtype: int64
****************************************************************************************************
Numerical Test missing Values 
LotFrontage    84
GarageYrBlt    28
MasVnrArea      1
dtype: int64


In [0]:
df_train.fillna(df_train.mean(), inplace=True)
df_test.fillna(df_test.mean(), inplace=True)

'''
After filling the empty numerical features with mean of their columns
'''
_numerical_missing_values_train = df_train[numerical_features].isnull().sum().sort_values(ascending=False)
_numerical_missing_values_test = df_test[numerical_features].isnull().sum().sort_values(ascending=False)

_numerical_missing_values_train = _numerical_missing_values_train[_numerical_missing_values_train > 0]

_numerical_missing_values_test = _numerical_missing_values_test[_numerical_missing_values_test > 0]

print('Numerical Train missing Valuse ', _numerical_missing_values_train, sep='\n')

print('*'*100)

print('Numerical Test missing Values ', _numerical_missing_values_test, sep='\n')

Numerical Train missing Valuse 
Series([], dtype: int64)
****************************************************************************************************
Numerical Test missing Values 
Series([], dtype: int64)


Datas empty and NaN are removed accordingly, 


In [0]:
df_train.describe()

MSSubClass  LotFrontage  ...       YrSold      SalePrice
count  1000.000000  1000.000000  ...  1000.000000    1000.000000
mean     55.435000    71.310303  ...  2007.793000  182854.640000
std      41.477779    21.965542  ...     1.329631   82290.996386
min      20.000000    21.000000  ...  2006.000000   34900.000000
25%      20.000000    60.000000  ...  2007.000000  130000.000000
50%      50.000000    71.310303  ...  2008.000000  163950.000000
75%      70.000000    80.000000  ...  2009.000000  215000.000000
max     190.000000   313.000000  ...  2010.000000  755000.000000

[8 rows x 37 columns]

In [0]:
df_test.describe()

MSSubClass  LotFrontage  ...      MoSold       YrSold
count  460.000000   460.000000  ...  460.000000   460.000000
mean    60.076087    67.284574  ...    6.330435  2007.865217
std     43.914463    21.968017  ...    2.743093     1.324845
min     20.000000    21.000000  ...    1.000000  2006.000000
25%     20.000000    60.000000  ...    5.000000  2007.000000
50%     50.000000    67.284574  ...    6.000000  2008.000000
75%     70.000000    75.000000  ...    8.000000  2009.000000
max    190.000000   313.000000  ...   12.000000  2010.000000

[8 rows x 36 columns]

# Encoding the Categorical

Resource Used : [Encoding Resource](https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02)

In [0]:
for i in categorical_features:
    mean_encode = df_train.groupby(i)['SalePrice'].mean()
    print(mean_encode)
    print('*'*10)
    df_train.loc[:,i+'_enc'] = df_train[i].map(mean_encode)
    df_train.drop(i, axis=1)
    df_test.loc[:, i + '_enc'] = df_test[i].map(mean_encode)

MSZoning
C (all)     79509.857143
FV         216808.315789
RH         126458.333333
RL         191704.449132
RM         131591.744526
Name: SalePrice, dtype: float64
**********
Street
Grvl    132528.600000
Pave    183107.534673
Name: SalePrice, dtype: float64
**********
Alley
Grvl    125367.461538
None    184725.155789
Pave    171091.166667
Name: SalePrice, dtype: float64
**********
LotShape
IR1    206925.344311
IR2    261356.416667
IR3    216151.666667
Reg    166704.037915
Name: SalePrice, dtype: float64
**********
LandContour
Bnk    143836.475000
HLS    234550.472222
Low    207991.666667
Lvl    181850.626667
Name: SalePrice, dtype: float64
**********
Utilities
AllPub    182900.04004
NoSeWa    137500.00000
Name: SalePrice, dtype: float64
**********
LotConfig
Corner     184875.136842
CulDSac    228405.750000
FR2        173067.703704
FR3        208475.000000
Inside     178744.493741
Name: SalePrice, dtype: float64
**********
LandSlope
Gtl    181426.262936
Mod    204413.456522
Sev    234

In [0]:
new_numerical_features = df_train.dtypes[df_train.dtypes != 'object'].index
print(len(new_numerical_features))

80


## Correlation

In [0]:
abs_correlation = df_train.corr().abs()
correlation_values = abs_correlation.nlargest(len(abs_correlation), 'SalePrice')['SalePrice']
print(correlation_values)

SalePrice           1.000000
OverallQual         0.791954
Neighborhood_enc    0.737620
GrLivArea           0.721662
ExterQual_enc       0.690006
                      ...   
MoSold              0.029087
LowQualFinSF        0.024810
Utilities_enc       0.017446
BsmtFinSF2          0.009186
BsmtHalfBath        0.006024
Name: SalePrice, Length: 80, dtype: float64


Picking Columns with Good Correlation say 0.45

In [0]:
minimum_correlation = 0.45
good_correlation_cols = list(correlation_values[correlation_values.values >= minimum_correlation].index)
print('Good Correlation Columns ', '\n'.join(good_correlation_cols), sep='\n')
print('Effective Columns after correlation ', len(good_correlation_cols))

Good Correlation Columns 
SalePrice
OverallQual
Neighborhood_enc
GrLivArea
ExterQual_enc
KitchenQual_enc
BsmtQual_enc
GarageCars
TotalBsmtSF
1stFlrSF
GarageArea
FullBath
FireplaceQu_enc
TotRmsAbvGrd
GarageFinish_enc
YearBuilt
Foundation_enc
YearRemodAdd
GarageType_enc
Fireplaces
GarageYrBlt
MasVnrArea
BsmtFinType1_enc
Effective Columns after correlation  23


In [0]:
'''
drop columns with less correlation
'''
df_train = df_train[good_correlation_cols]
good_correlation_cols_test = good_correlation_cols[:]
good_correlation_cols_test.remove('SalePrice')
df_test = df_test[good_correlation_cols_test]

### Decision Tree

Decision Tree - Regression
Decision tree builds regression or classification models in the form of a tree structure. It breaks down a dataset into smaller and smaller subsets while at the same time an associated decision tree is incrementally developed. The final result is a tree with decision nodes and leaf nodes.

[Source](https://www.saedsayad.com/decision_tree_reg.htm)

In [0]:

class decision_tree_regression(object):
    def __init__(self, construction_metric='mse', max_depth=80):
        '''
        feature         ----    current node deciding feature
        target_value    ----    mean value of the target
        samples_count   ----    count the dataset associated with current node count
        gain            ----    current gain by selection this feature as tree node
        threshold       ----    split value
        cons_metric     ----    mse, mae
        max_depth       ----    tree max depth 
        '''
        self.feature = self.target_value = self.samples_count = self.gain = self.left = \
        self.right = self.threshold = None
        self.depth = 0
        self.root = None

        self.construction_metric = construction_metric
        
        self.max_depth = max_depth

    def fit(self, features, target):
        '''
        features --- features data frame
        target   --- vector of corresponding target values
        '''
        self.root = decision_tree_regression()
        self.root.__grow_tree(features, target, self.construction_metric)
        return 

    def __get_impurity(self, target, construction_metric):
        if 'mse':
            '''
            sum( (y-y_predict)^2 ) / len(target)
            '''
            return np.mean((target - np.mean(target)) ** 2)
        elif 'mae':
            '''
            sum ( abs(y - y_predict) ) / len(target)
            '''
            return np.mean(np.abs(target - np.mean(target)))

    def __split_tree(self, features, target, construction_metric):
        if self.threshold is None:
            return
        '''
        left split
        '''
        left_features = features[features[:, self.feature] <= self.threshold]
        left_target = target[features[:, self.feature] <= self.threshold]

        self.left = decision_tree_regression()
        self.left.depth = self.depth + 1
        self.left.__grow_tree(left_features, left_target, construction_metric)

        '''
        right split
        '''
        right_features = features[features[:, self.feature] > self.threshold]
        right_target = target[features[:,self.feature]>self.threshold]

        self.right = decision_tree_regression()
        self.right.depth = self.depth + 1
        self.right.__grow_tree(right_features, right_target, construction_metric)


    def __grow_tree(self, features, target, construction_metric='mse'):
        self.samples_count = features.shape[0]

        if len(np.unique(target)) == 1:
            self.target_value = target[0]
            return
        
        best_gain = 0
        best_threshold = potential_split_point = None

        self.target_value = np.mean(target)
        
        node_impurity = self.__get_impurity(target, construction_metric)

        for col in range(features.shape[1]):
            distinct_feature_values = np.unique(features[:, col])
            thresholds = (distinct_feature_values[:-1] + distinct_feature_values[1:]) * 0.5

            for threshold in thresholds:
                left_target_split = target[features[:,col] <= threshold]
                left_impurity = self.__get_impurity(left_target_split, construction_metric)

                right_target_split = target[features[:,col] > threshold]
                right_impurity = self.__get_impurity(right_target_split, construction_metric)

                _gain = node_impurity - (float(left_target_split.shape[0])/ self.samples_count * left_impurity + \
                float(right_target_split.shape[0])/ self.samples_count * right_impurity)

                if _gain > best_gain:
                    best_gain = _gain
                    potential_split_point = col
                    best_threshold = threshold
        
        self.feature = potential_split_point
        self.gain = best_gain
        self.threshold = best_threshold
        self.__split_tree(features, target, construction_metric)
        return

    def _predict(self, feature):
        if self.feature is not None:
            if feature[self.feature] < self.threshold:
                return self.left._predict(feature)
            else:
                return self.right._predict(feature)
        else:
            return self.target_value

    def predict(self, features):
        '''
        features --- dataframe of features
        predicts the target values for the features given as input
        '''
        return np.array([self.root._predict(feature) for feature in features])

### HELPER FUNCTIONS





In [0]:
def train_test_split(df, validation_size=100):
    random.seed(0)
    indicies = df.index.tolist()
    test_indicies = random.sample(population=indicies, k=validation_size)
    test_df = df.loc[test_indicies]
    train_df = df.drop(test_indicies)
    return train_df, test_df

In [0]:
df_train, df_val = train_test_split(df_train)

In [0]:
df_train_y = df_train['SalePrice']
df_train_x = df_train.drop('SalePrice', axis=1)

df_val_y = df_val['SalePrice']
df_val_x = df_val.drop('SalePrice', axis=1)

In [0]:
df_train_x.head()

OverallQual  Neighborhood_enc  ...  MasVnrArea  BsmtFinType1_enc
0            8     313885.875000  ...       388.0     239506.971326
2            6     162898.000000  ...       443.0     239506.971326
3            6     230471.884615  ...        76.0     239506.971326
4            5     134159.474576  ...         0.0     164140.746753
5            7     313885.875000  ...       106.0     174257.722222

[5 rows x 22 columns]

In [0]:
 df_val_x.head()

OverallQual  Neighborhood_enc  ...  MasVnrArea  BsmtFinType1_enc
864            7     230471.884615  ...       432.0     239506.971326
394            7     193299.360000  ...         0.0     174257.722222
776            2     140895.000000  ...         0.0     147262.531250
911            6     128989.078947  ...        80.0     239506.971326
430            5     132879.676923  ...         0.0     174257.722222

[5 rows x 22 columns]

In [0]:
decision_tree_regress = decision_tree_regression(construction_metric='mae', max_depth=24)
decision_tree_regress.fit(df_train_x.values, df_train_y.values)
predicted_result = decision_tree_regress.predict(df_val_x.values)

## Sklearn Decision Tree Regressor Comparision


In [0]:
sk_learn_regress = tree.DecisionTreeRegressor(criterion='mae')
sk_learn_regress.fit(df_train_x.values, df_train_y.values)
sk_learn_predict_result = sk_learn_regress.predict(df_val_x.values)

In [0]:
our_mse = mean_squared_error(predicted_result, df_val_y)
our_mae = mean_absolute_error(predicted_result, df_val_y)
from sklearn.metrics import r2_score
our_r2 = r2_score(predicted_result, df_val_y)
sk_learn_r2 = r2_score(predicted_result, df_val_y)

print('R2 Score ', our_r2, 'Sk learn ', sk_learn_r2)

sk_learn_mse = mean_squared_error(sk_learn_predict_result, df_val_y)
sk_learn_mae = mean_absolute_error(sk_learn_predict_result, df_val_y)

print('OUR MSE ',math.sqrt(our_mse), 'Sk Learn MSE', math.sqrt(sk_learn_mse))

print('OUR MAE ', our_mae, 'Sk Learn MAE', sk_learn_mae)

print('Our r2 ', )

R2 Score  0.5715258087782702 Sk learn  0.5715258087782702
OUR MSE  56408.80286400342 Sk Learn MSE 49790.28379242279
OUR MAE  33442.95 Sk Learn MAE 30236.73
Our r2 


In [0]:
act_test_values = pd.read_csv('test_labels.csv', names=['Id', 'SalePrice']).values
act_test_values = act_test_values[:, 1]

In [0]:
our_predict_act_test = decision_tree_regress.predict(df_test.values)
sk_learn_predict_act_test = sk_learn_regress.predict(df_test.values)
print('SKlearn Actual Test Result MSE', math.sqrt(mean_squared_error(sk_learn_predict_act_test, act_test_values)), 'Our Actual Test Result MSE ',math.sqrt(mean_squared_error(our_predict_act_test, act_test_values)))

print('SKlearn Actual Test Result MAE', (mean_absolute_error(sk_learn_predict_act_test, act_test_values)), 'Our Actual Test Result MAE ',(mean_absolute_error(our_predict_act_test, act_test_values)))

SKlearn Actual Test Result MSE 40444.06449994964 Our Actual Test Result MSE  38161.90975396945
SKlearn Actual Test Result MAE 27637.815217391304 Our Actual Test Result MAE  26210.483695652172


# Data Analysis

***Skewness*** : Skewness is a measure of symmetry, or more precisely, the lack of symmetry. A distribution, or data set, is symmetric if it looks the same to the left and right of the center point.

Dealing with Skewness of curve :
[link text](https://www.researchgate.net/post/how_to_deal_with_skewness)

In [0]:
'''
Skewness of Target Variable
'''
sb.distplot(df['label'])

print('Skewness ' + str(df['label'].skew()))

In [0]:
df['label'] = np.log(df['label'])
sb.distplot(df['label'])
print('Skewness ' + str(df['label'].skew()))